# Hyperparameter Tuning using HyperDrive


In [ ]:
from azureml.core import Workspace, Environment
from azureml.core import Experiment
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import Rundetails

from azureml.train.hyperdrive import BayesianParameterSampling, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import normal, uniform, choice

import shutil
import os

## Dataset

In [ ]:
data_path = "https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset"

ibm_ds = Dataset.Tabular.from_delimited_files(path=data_path)

In [ ]:
ws = Workspace.from_config()
experiment_name = 'dnn-experiment'

experiment=Experiment(ws, experiment_name)

## Compute Target

In [ ]:
cluster_name = "hd-dnn-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Reusing already created compute target")

except ComputeTargetException:
    print("Creating compute target...")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6", max_nodes=6)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

In [ ]:
project_folder = "./dnn-attrition"
os.makedirs(project_folder, exist_ok=True)
shutil.copy("tabular-dnn.py", project_folder)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# early_termination_policy = <your policy here>

param_sampling = BayesianParameterSampling({
    "learning_rate": uniform(0.001, 0.1),
    "num_epochs": choice(range(100, 5000)),
    "layer1": choice(range(10, 100)),
    "layer2": choice(range(10, 100)),
})

conda_env = Environment.get(workspace=ws, name="AzureML-PyTorch-1.6-GPU")
# conda_env.add_conda_package("scikit-learn")
# Specify a GPU base image
# pytorch_env.docker.enabled = True
# pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

src = ScriptRunConfig(source_directory=project_folder,
                      script="dnn-train.py",
                      arguments=["--num_epochs", 100, "--layer1", 30, "--layer2", 15, "--learning_rate", 0.001,"--output_dir", "./outputs"],
                      compute_target=compute_target,
                      envionment=conda_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src, 
                                         hyperparameter_sampling=param_sampling, 
                                         primary_metric_name='best_test_acc', 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=8,
                                         max_concurrent_runs=4)

In [ ]:
run = experiment.submit(src)
print(run)

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status == "Completed")

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)


In [ ]:

print('Best Run is:\n  Validation accuracy: {0:.5f} \n  Learning rate: {1:.5f} \n Layer 1: {} \n layer 2: {}'.format(
        best_run_metrics['best_test_acc'][-1],
        best_run_metrics['learning_rate'],
        best_run_metrics['layer1'],
        best_run_metrics['layer2'])
     )

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='pytorch-attrition', 
                                model_path='outputs/dnn-model.pt')
print(model.name, model.id, model.version, sep='\t')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service